# Background Information

In this notebook I will set up a basic flow for uploading and dowloading files. Formating datasets and ultimately running cytetype.

In [1]:
# Importing Require Libraries
import cytetype
import scanpy as sc
import session_info
import pandas as pd
import os
import numpy as np
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
# Session Info
session_info.show()

/Users/alexantill/miniconda3/envs/benchLLM/lib/python3.12/site-packages/session_info/main.py:213: UserWarning: The '__version__' attribute is deprecated and will be removed in MarkupSafe 3.1. Use feature detection, or `importlib.metadata.version("markupsafe")`, instead.
  mod_version = _find_version(mod.__version__)


In [4]:
# Importing AnnData file
# File Path
#file_path_Simone = os.path.expanduser("~/Göran_Karlsson_Lab/benchLLM/Simone_et_al/7d821d98-5b42-4480-8173-641c1b37b237.h5ad")
file_path_Granja = os.path.expanduser("~/Göran_Karlsson_Lab/benchLLM/Granja_et_al/scRNA-Hematopoiesis-Granja-2019.h5ad")
#file_path_Jardine = os.path.expanduser("~/Göran_Karlsson_Lab/benchLLM/Jardine_et_al/fig1b_fbm_scaled_gex_updated_dr_20210104.h5ad")

# AnnData Read In
#adata = sc.read_h5ad(file_path_Simone)
adata = sc.read_h5ad(file_path_Granja)

In [5]:
# Normalize (10,000 Counts per Cell)
sc.pp.normalize_total(adata, target_sum=1e4)

# Log1p transform
sc.pp.log1p(adata)

In [26]:
# Explore log1p Error (It is due to NaNs)
print("Min:", np.min(adata.X))
print("Any NaNs in sparse data?", np.isnan(adata.X.data).any())
print("Any Infs in sparse data?", np.isinf(adata.X.data).any())

Min: nan
Any NaNs in sparse data? True
Any Infs in sparse data? False


In [5]:
# Find Most Up/Down Regulated Genes 
sc.tl.rank_genes_groups(adata, groupby='cell_type', method='t-test', key_added = 'Rank_Genes_cell_type')  # try 'wilcoxon' if accurate

In [6]:
# Find Most Up/Down Regulated Genes 
sc.tl.rank_genes_groups(adata, groupby='BioClassification', method='t-test', key_added = 'Rank_Genes_cell_type')  # try 'wilcoxon' if accurate

/Users/alexantill/miniconda3/envs/benchLLM/lib/python3.12/site-packages/scanpy/tools/_rank_genes_groups.py:456: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, "names"] = self.var_names[global_indices]
/Users/alexantill/miniconda3/envs/benchLLM/lib/python3.12/site-packages/scanpy/tools/_rank_genes_groups.py:458: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, "scores"] = scores[global_indices]
/Users/alexantill/miniconda3/envs/benchLLM/lib/python3.12/site-packages/scanpy/tools/_rank_gene

In [7]:
annotator = cytetype.CyteType(adata, group_key = 'cell_type', rank_key= 'Rank_Genes_cell_type', gene_symbols_column = 'gene_name')

KeyError: "Cell group key 'cell_type' not found in `adata.obs`."

In [11]:
annotator = cytetype.CyteType(adata, group_key = 'BioClassification', rank_key= 'Rank_Genes_cell_type', gene_symbols_column = 'name')

Calculating expression percentages.
Extracting marker genes.
Data preparation completed. Ready for submitting jobs.


In [44]:
# Run annotation with custom model
adata = annotator.run(
    study_context = "Human adult mixed-phenotype acute leukemias from peripheral blood and bone marrow, analysed 10x genomics single cell RNA-seq kits",
    model_config = [{
        'provider': 'openai',
        'name': 'qwen/qwen3-235b-a22b',
        'apiKey': os.environ.get('OPENROUTER_API_KEY'),
        'baseUrl': "https://openrouter.ai/api/v1",
    }],
    auth_token = os.environ.get('NYGEN_API_KEY'),
    timeout_seconds=3000,
    run_config={
        'concurrentClusters': 5,        # Default: 3, Range: 2-10
        'maxAnnotationRevisions': 2,    # Default: 2, Range: 1-5
        'maxLLMRequests': 2000           # Default: 500, Range: 50-2000
    },
)

Waiting for results for job ID: b56a9a5a-0909-4084-a435-9524298b875c
View the automatically updating visualization report at: https://nygen-labs-prod--cell-annotation-agent-fastapi-app.modal.run/report/b56a9a5a-0909-4084-a435-9524298b875c
Could not fetch logs for job b56a9a5a-0909-4084-a435-9524298b875c: 404 Client Error: Not Found for url: https://nygen-labs-prod--cell-annotation-agent-fastapi-app.modal.run/logs/b56a9a5a-0909-4084-a435-9524298b875c
250602:1522:44 |INFO| [STARTING WORKFLOWS]: Job ID: b56a9a5a-0909-4084-a435-9524298b875c
250602:1522:44 |INFO| [WORKFLOW STEP 1/3]: Generating context summary
250602:1524:00 |INFO| [WORKFLOW STEP 2/3]: Running cluster contextualization, annotation, review, and ontology assignment
250602:1524:03 |INFO| [Cluster: 1] Generating cluster-specific context...
250602:1524:06 |INFO| [Cluster: 10] Generating cluster-specific context...
250602:1524:09 |INFO| [Cluster: 11] Generating cluster-specific context...
250602:1524:12 |INFO| [Cluster: 12] Gener

KeyboardInterrupt: 

In [39]:
Qwen = adata.uns['cytetype_results']['result']

In [40]:
Author_vs_Qwen3 = pd.crosstab(adata.obs.BioClassification, adata.obs.cytetype_BioClassification).T.idxmax()

In [ ]:
Author_vs_Qwen3.reset_index().to_csv("/Users/alexantill/Göran_Karlsson_Lab/benchLLM/Granja_et_al/author_vs_Qwen3.csv", index=False)

In [ ]:
models = [
    "google/gemini-2.5-pro-preview",
    "anthropic/claude-3.7-sonnet:thinking",
    "openai/o3-mini",
    "openai/o4-mini",
    "x-ai/grok-3-beta",
    "meta-llama/llama-4-maverick",
    "anthropic/claude-3.7-sonnet",
    "openai/gpt-4.1-mini",
    "qwen/qwen3-235b-a22b",
    "google/gemini-2.0-flash-001",
    "openai/gpt-4.1"
]

results = {}

for model_full in models:
    model_name = model_full.split("/")[-1]
    adata_result = annotator.run(
        study_context="Healthy adult human peripheral blood, analysed on a variety of commerical single cell RNA-seq kits",
        model_config=[{
            'provider': 'openai',  # update this if provider changes!
            'name': model_full,
            'apiKey': os.environ.get('OPENROUTER_API_KEY'),
            'baseUrl': "https://openrouter.ai/api/v1",
        }],
        auth_token=os.environ.get('NYGEN_API_KEY'),
        timeout_seconds=1500,
    )
    results[f'adata_{model_name}'] = adata_result

    # Create crosstab and get idxmax
    cross_tab = pd.crosstab(
        adata_result.obs.cell_type,
        adata_result.obs.cytetype_cell_type
    ).T.idxmax()

    cross_tab_df = cross_tab.reset_index()
    
    # Save to CSV with model-specific filename
    output_path = f"/Users/alexantill/Göran_Karlsson_Lab/benchLLM/Simone_et_al/author_vs_{model_name}.csv"
    cross_tab_df.to_csv(output_path, index=False)
    print(f"Saved crosstab for {model_name} to {output_path}")
